# Image Classification on MNIST

In this notebook we demonstrate how to build a neural network to classify images. The neural network will be built using the Keras style API available in Analytics Zoo, and use a BigDL optimizer to train the model.

Data: 
* The dataset used is MNIST, which is a collection of handwritten numbers ranging from 0 to 9. The data training dataset contains 60000 images and the test set contains 10000 images. We will try to predict the number in each image.

## Intialization

* import necessary libraries

In [1]:
# Copyright 2018 Analytics Zoo Authors.
# http://www.apache.org/licenses/LICENSE-2.0

import warnings
warnings.filterwarnings("ignore", message = "numpy.dtype size changed")

import tensorflow as tf
from zoo import init_nncontext
from zoo.pipeline.api.net import TFOptimizer, TFDataset, TFPredictor
from bigdl.optim.optimizer import *
import sys
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from bigdl.dataset import mnist
from bigdl.dataset.transformer import *

import mnist_utilities_workshop

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


* Initilaize NN context, it will get a SparkContext with optimized configuration for BigDL performance.

In [2]:
sc = init_nncontext()

## Data Preparation

* Create function to import and format the images.

In [5]:
# get data, pre-process and create TFDataset
def get_data_rdd(dataset):
    (images_data, labels_data) = mnist_utilities_workshop.ws_read_data_sets("/opt/work/ODSC-east-2019/datasets/mnist/", dataset)
    # image_rdd = sc.parallelize(images_data[:data_num])
    image_rdd = sc.parallelize(images_data)
    # labels_rdd = sc.parallelize(labels_data[:data_num])
    labels_rdd = sc.parallelize(labels_data)
    rdd = image_rdd.zip(labels_rdd) \
        .map(lambda rec_tuple: [normalizer(rec_tuple[0], mnist.TRAIN_MEAN, mnist.TRAIN_STD),
                                np.array(rec_tuple[1])])
    return rdd

* Download and read the MNIST data

In [6]:
training_rdd = get_data_rdd("train")
testing_rdd = get_data_rdd("test")

('Extracting', '/home/bala/odsc/ODSC-east-2019/datasets/mnist/train-images-idx3-ubyte.gz')
('Extracting', '/home/bala/odsc/ODSC-east-2019/datasets/mnist/train-labels-idx1-ubyte.gz')
('Extracting', '/home/bala/odsc/ODSC-east-2019/datasets/mnist/t10k-images-idx3-ubyte.gz')
('Extracting', '/home/bala/odsc/ODSC-east-2019/datasets/mnist/t10k-labels-idx1-ubyte.gz')


* Format the RDDs into a TensorFlow Dataset object. Format the shape of the input features to that of the images.

In [7]:
# Batch size can be adjusted to improve the model, but must be a multiple of the total number of cores in the Spark environment
batch_size = 512

# Images are 28 by 28 pixels with one color channel for black and white; labels have no dimmensions because they are integers
dataset = TFDataset.from_rdd(training_rdd,
                             names=["features", "labels"],
                             shapes=[[28, 28, 1], []], 
                             types=[tf.float32, tf.int32],
                             batch_size=batch_size,
                             val_rdd=testing_rdd
                             )
pred_dataset = TFDataset.from_rdd(training_rdd,
                                  names=["features", "labels"],
                                  shapes=[[28, 28, 1], []], 
                                  types=[tf.float32, tf.int32],
                                  batch_per_thread=1)

## Build Model

* Create the model structure, with inputs being the shape of the images and outputs being the number of image classes.

In [8]:
data = Input(shape=[28, 28, 1]) # Must match the dimmensions of the images specified in the TFDataset
# x = Convolution2D(6, 5, 5, activation='tanh', name='conv1_5x5')(data)
# x = MaxPooling2D()(x)
x = Flatten()(data)
x = Dense(64, activation='relu')(x) # The number of dense units can be adjusted to attempt to improve the model.
x = Dense(64, activation='relu')(x)
 # The number of dense units in the predictions (output) layer should match the number of classes
predictions = Dense(10, activation='softmax')(x)

## Initialize the model and the optimizer.

In [9]:
model = Model(inputs=data, outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

optimizer = TFOptimizer.from_keras(keras_model=model, dataset=dataset)

creating: createRMSprop
creating: createClassNLLCriterion
creating: createLoss
creating: createZooKerasAccuracy
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
creating: createTFTrainingHelper
creating: createTFValidationMethod
creating: createTFValidationMethod
creating: createIdentityCriterion
creating: createMaxEpoch
creating: createDistriOptimizer
creating: createEveryEpoch


## Set location for model training summary output.
## Train the model.

In [10]:
optimizer.set_train_summary(TrainSummary("/tmp/mnist_log", "mnist"))
optimizer.set_val_summary(ValidationSummary("/tmp/mnist_log", "mnist"))
max_epoch = 15
optimizer.optimize(end_trigger=MaxEpoch(max_epoch))

creating: createTrainSummary
creating: createValidationSummary
creating: createMaxEpoch


## Save the weights so it can be read later for inference

In [11]:
model.save_weights("mnist_keras.h5")

## Run Tensorboard

tensorboard --logdir=/tmp/mnist_log/mnist/train --port 8082